In [1]:
%reload_ext autoreload
%autoreload 2

import sys
from pathlib import Path 
current_path = Path().resolve()
abs_path = str(current_path.parent)
sys.path.append(abs_path)

import warnings
warnings.filterwarnings('ignore')

import numpy as np
import pandas as pd
pd.set_option('display.max_columns', None)

from datetime import datetime as dt
import pytz
import ast
import operator

RAW_PATH = current_path.parent / 'data' / 'raw'
OUTPUT_PATH = current_path.parent / 'data' / 'output'

In [2]:
import scripts.waze_flood as wf

[DEBUG] Loading variable profile from defaults.
[DEBUG] Loading variable config_file from defaults.
[DEBUG] Loading variable credentials_file from defaults.
[DEBUG] Loading variable data_path from defaults.
[DEBUG] Loading variable profile from defaults.
[DEBUG] Loading variable ca_bundle from defaults.
[DEBUG] Loading variable profile from defaults.
[DEBUG] Loading variable api_versions from defaults.
[DEBUG] Loading variable profile from defaults.
[DEBUG] Loading variable credentials_file from defaults.
[DEBUG] Loading variable config_file from defaults.
[DEBUG] Loading variable profile from defaults.
[DEBUG] Loading variable metadata_service_timeout from defaults.
[DEBUG] Loading variable profile from defaults.
[DEBUG] Loading variable metadata_service_num_attempts from defaults.
[DEBUG] Loading variable profile from defaults.
[DEBUG] Looking for credentials via: env
[DEBUG] Looking for credentials via: assume-role
[DEBUG] Looking for credentials via: shared-credentials-file
[INFO] 

# Downloading Data

## Donwloading São Paulo Enchentes

Os dados de enchente foram obtidos através de query feita no Athena e importados como arquivo csv

In [3]:
df = pd.read_csv(RAW_PATH/"sp-19-02-04-enchentes.csv")

## Time Range

In [4]:
wf.get_timerange(df)

0    2019-02-03 21:57:00.000 America/Sao_Paulo
1    2019-02-04 21:56:00.000 America/Sao_Paulo
dtype: object

# Data pre-processing

## Clustering Enchentes

Clusterizou os alertas de enchentes através do método st_dbscan

In [5]:
df['start_time'] = df['start_time'].apply(lambda x: dt.strptime(x, '%Y-%m-%d %H:%M:%S.000 America/Sao_Paulo'))
df['end_time'] = df['end_time'].apply(lambda x: dt.strptime(x, '%Y-%m-%d %H:%M:%S.000 America/Sao_Paulo'))

In [6]:
clusters_dict = wf.clustering(df, eps_spatial=200, eps_temporal=3600)

In [7]:
clusters = pd.DataFrame(clusters_dict).T.drop(columns=['uuid list']).rename_axis('cluster')

In [8]:
clusters['interactions'] = clusters['thumbs_up'] + clusters['uuid count']

## Floods to shapely

In [9]:
wf.treat_alert_points(clusters)

In [10]:
clusters.head(1)

,confidence_max,confidence_min,end_time,location_x,location_y,reliability_max,reliability_min,start_time,thumbs_up,uuid count,interactions,point
cluster,,,,,,,,,,,,
1,0,0,2019-02-04 05:32:00,-46.5978,-23.589,5,5,2019-02-04 04:32:00,0,1,1,POINT (-46.59784599999999 -23.589049)


# Ranking Clusters by Interactions

In [11]:
total_interactions = clusters['interactions'].sum()
total_alerts = clusters['uuid count'].sum()

In [12]:
total_interactions

1720

In [13]:
total_alerts

559

In [14]:
clusters['pareto'] = clusters['interactions'].sort_values(ascending=False).cumsum() * 100 / total_interactions

In [15]:
clusters_pareto_sorted = pd.DataFrame(clusters.sort_values(by='pareto'))
clusters_pareto_sorted.head()

,confidence_max,confidence_min,end_time,location_x,location_y,reliability_max,reliability_min,start_time,thumbs_up,uuid count,interactions,point,pareto
cluster,,,,,,,,,,,,,
21,5,0,2019-02-04 11:53:00,-46.63,-23.5195,10,5,2019-02-04 02:36:00,83,31,114,POINT (-46.629964964799 -23.51946167009796),6.62791
8,4,0,2019-02-04 15:33:00,-46.5592,-23.5104,10,5,2019-02-04 06:25:00,31,41,72,POINT (-46.55923441955649 -23.51041344798837),10.814
67,5,0,2019-02-04 07:35:00,-46.6933,-23.5978,10,5,2019-02-04 04:55:00,54,3,57,POINT (-46.69331008391696 -23.59779010679273),14.1279
186,5,0,2019-02-04 10:20:00,-46.693,-23.6143,10,5,2019-02-04 04:54:00,45,6,51,POINT (-46.69295849999998 -23.61429099999999),17.093
192,5,0,2019-02-04 09:14:00,-46.6758,-23.6271,10,5,2019-02-04 05:29:00,39,2,41,POINT (-46.67576243220336 -23.62707160169494),19.4767


In [16]:
clusters_pareto_sorted.to_csv(OUTPUT_PATH/'clusters_pareto.csv')

# Bounding-Box Macro Regiões

Cada bounding box é construído com um ponto de alerta limitando cada lado.

In [17]:
def generate_bbox(clusters, cluster_lower_y, cluster_left_x, cluster_upper_y, cluster_right_x):
    bbox = {}
    bbox['lower_y'] = clusters.loc[cluster_lower_y, 'location_y']
    bbox['left_x'] = clusters.loc[cluster_left_x, 'location_x']
    bbox['upper_y'] = clusters.loc[cluster_upper_y, 'location_y']
    bbox['rigth_x'] = clusters.loc[cluster_right_x, 'location_x']
    return bbox

In [18]:
def clusters_in_bbox(clusters, bbox):
    lower_y = bbox['lower_y']
    left_x = bbox['left_x']
    upper_y = bbox['upper_y']
    rigth_x = bbox['rigth_x']
    return clusters[(left_x <= clusters['location_x']) & (clusters['location_x'] <= rigth_x) & (lower_y <= clusters['location_y']) & (clusters['location_y'] <= upper_y)]

Caso venha a ser feito novo relatório baseado neste, recomenda-se uma list ou dict de bbox, sendo cada entrada uma bbox de uma região para evitar criar variáveis com nomes reduntantes (bbox1, bbox2, bbox3,...)

## Região 1 (zona Norte/Leste - Tiete)

In [19]:
# lower_y = cluster 44
# left_x = cluster 64
# upper_y = cluster 45
# rigth_x = cluster 166

In [20]:
bbox1 = generate_bbox(clusters, 44,64,45,166)

In [21]:
clusters_reg1 = clusters_in_bbox(clusters, bbox1)

In [22]:
total_interactions_reg1 = clusters_reg1['interactions'].sum()
print(total_interactions_reg1)
print(total_interactions_reg1 / total_interactions *100)

342
19.883720930232556


In [23]:
clusters_reg1.to_csv(OUTPUT_PATH/'clusters_reg1.csv')

## Região 2 (zona Oeste - Pinheiros)

In [24]:
# lower_y = cluster 186
# left_x = cluster 178
# upper_y = cluster 171
# rigth_x = cluster 186

In [25]:
bbox2 = generate_bbox(clusters, 186, 178, 171, 186)

In [26]:
clusters_reg2 = clusters_in_bbox(clusters, bbox2)

In [27]:
total_interactions_reg2 = clusters_reg2['interactions'].sum()

In [28]:
print(total_interactions_reg2)
print(total_interactions_reg2 / total_interactions *100)

149
8.662790697674419


In [29]:
clusters_reg2.to_csv(OUTPUT_PATH/'clusters_reg2.csv')

## Região 3 (Av. Jornalista Roberto Marinho)

In [30]:
# lower_y = cluster 141
# left_x = cluster 47
# upper_y = cluster 47
# rigth_x = cluster 141

In [31]:
bbox3 = generate_bbox(clusters, 141, 47, 47, 141)

In [32]:
clusters_reg3 = clusters_in_bbox(clusters, bbox3)

In [33]:
total_interactions_reg3 = clusters_reg3['interactions'].sum()

In [34]:
print(total_interactions_reg3)
print(total_interactions_reg3 / total_interactions *100)

51
2.9651162790697674


## Região 4 (Rua Miguel Yunes)

In [35]:
# lower_y = cluster 77
# left_x = cluster 39
# upper_y = cluster 39
# rigth_x = cluster 77

In [36]:
bbox4 = generate_bbox(clusters, 77, 39, 39, 77)

In [37]:
clusters_reg4 = clusters_in_bbox(clusters, bbox4)

In [38]:
total_interactions_reg4 = clusters_reg4['interactions'].sum()

In [39]:
print(total_interactions_reg4)
print(total_interactions_reg4 / total_interactions *100)

87
5.058139534883721


In [40]:
clusters_reg4.to_csv(OUTPUT_PATH/'clusters_reg4.csv')

## Região 5 (Centro)

In [41]:
# lower_y = cluster 150
# left_x = cluster 66
# upper_y = cluster 66
# rigth_x = cluster 159

In [42]:
bbox5 = generate_bbox(clusters, 150, 66, 66, 159)

In [43]:
clusters_reg5 = clusters_in_bbox(clusters, bbox5)

In [44]:
total_interactions_reg5 = clusters_reg5['interactions'].sum()

In [45]:
print(total_interactions_reg5)
print(total_interactions_reg5 / total_interactions *100)

61
3.5465116279069764


In [46]:
clusters_reg5.to_csv(OUTPUT_PATH/'clusters_reg5.csv')

## Região 6 (Rodovia Anchieta)

In [47]:
# lower_y = cluster 6
# left_x = cluster 217
# upper_y = cluster 217
# rigth_x = cluster 3

In [48]:
bbox6 = generate_bbox(clusters, 6, 217, 217, 3)

In [49]:
clusters_reg6 = clusters_in_bbox(clusters, bbox6)

In [50]:
total_interactions_reg6 = clusters_reg6['interactions'].sum()

In [51]:
print(total_interactions_reg6)
print(total_interactions_reg6 / total_interactions *100)

34
1.9767441860465116


In [52]:
clusters_reg6.to_csv(OUTPUT_PATH/'clusters_reg6.csv')

## Região 7 (Av. Paulo Freire + Av. Condessa Elizabeth de Robiano)

In [53]:
# lower_y = cluster 36
# left_x = cluster 117
# upper_y = cluster 8
# rigth_x = cluster 36

In [54]:
bbox7 = generate_bbox(clusters, 36, 117, 8, 36)

In [55]:
clusters_reg7 = clusters_in_bbox(clusters, bbox7)

In [56]:
total_interactions_reg7 = clusters_reg7['interactions'].sum()

In [57]:
print(total_interactions_reg7)
print(total_interactions_reg7 / total_interactions *100)

165
9.593023255813954


In [58]:
clusters_reg7.to_csv(OUTPUT_PATH/'clusters_reg7.csv')

## Região 8 (Av. Doutor Luís Ayres)

In [59]:
# lower_y = cluster 74
# left_x = cluster 132
# upper_y = cluster 132
# rigth_x = cluster 74

In [60]:
bbox8 = generate_bbox(clusters, 74, 132, 132, 74)

In [61]:
clusters_reg8 = clusters_in_bbox(clusters, bbox8)

In [62]:
total_interactions_reg8 = clusters_reg8['interactions'].sum()

In [63]:
print(total_interactions_reg8)
print(total_interactions_reg8 / total_interactions *100)

40
2.3255813953488373


In [64]:
clusters_reg8.to_csv(OUTPUT_PATH/'clusters_reg8.csv')

## Região 3 (Av. Jornalista Roberto Marinho)

In [65]:
# lower_y = cluster 141
# left_x = cluster 47
# upper_y = cluster 47
# rigth_x = cluster 141

In [66]:
bbox9 = generate_bbox(clusters, 141, 47, 47, 141)

In [67]:
clusters_reg9 = clusters_in_bbox(clusters, bbox9)

In [68]:
total_interactions_reg9 = clusters_reg9['interactions'].sum()

In [69]:
print(total_interactions_reg9)
print(total_interactions_reg9 / total_interactions *100)

51
2.9651162790697674


In [70]:
clusters_reg9.to_csv(OUTPUT_PATH/'clusters_reg9.csv')

# Irregularities and Floods

A Metodologia para determinar o impacto de região com enchentes em uma via é observar se algum ponto da via marcado com trânsito irregular está contido nessa região

In [71]:
def point_in_bbox(x,y, bbox):
    lower_y = bbox['lower_y']
    left_x = bbox['left_x']
    upper_y = bbox['upper_y']
    rigth_x = bbox['rigth_x']
    return (left_x <= x) & (x <= rigth_x) & (lower_y <= y) & (y <= upper_y)

In [72]:
def line_intersect_bbox(line_str, bbox):
    line_dict = ast.literal_eval(line_str)
    coords = line_dict['coordinates']
    for point in coords:
        x = point[0]
        y = point[1]
        if point_in_bbox(x,y, bbox):
            return True
    return False

In [73]:
irreg_data = pd.read_csv(RAW_PATH/"irregularities.csv")

In [74]:
irreg_data['bbox1'] = irreg_data['irre_geojson'].apply(lambda x: line_intersect_bbox(x, bbox1))

In [75]:
irreg_data['bbox2'] = irreg_data['irre_geojson'].apply(lambda x: line_intersect_bbox(x, bbox2))

In [76]:
irreg_data['bbox3'] = irreg_data['irre_geojson'].apply(lambda x: line_intersect_bbox(x, bbox3))

In [77]:
irreg_data['bbox4'] = irreg_data['irre_geojson'].apply(lambda x: line_intersect_bbox(x, bbox4))

In [78]:
irreg_data['bbox5'] = irreg_data['irre_geojson'].apply(lambda x: line_intersect_bbox(x, bbox5))

In [79]:
irreg_data['bbox6'] = irreg_data['irre_geojson'].apply(lambda x: line_intersect_bbox(x, bbox6))

In [80]:
irreg_data['bbox7'] = irreg_data['irre_geojson'].apply(lambda x: line_intersect_bbox(x, bbox7))

In [81]:
irreg_data['bbox8'] = irreg_data['irre_geojson'].apply(lambda x: line_intersect_bbox(x, bbox8))

In [82]:
irreg_data.head()

,id,min_hour,max_hour,street,length,nthumbsup,driverscount,irre_geojson,bbox1,bbox2,bbox3,bbox4,bbox5,bbox6,bbox7,bbox8
0,162402419,11,11,Marg. Pinheiros,5899,59,3050,"{""type"":""LineString"", ""coordinates"":[[-46.7272...",False,False,False,False,False,False,False,False
1,162401047,8,8,SP-348 S Rod. dos Bandeirantes,3846,57,1007,"{""type"":""LineString"", ""coordinates"":[[-46.7406...",False,False,False,False,False,False,False,False
2,162406068,17,17,SP-280 O Rod. Pres. Castello Branco,6678,45,343,"{""type"":""LineString"", ""coordinates"":[[-46.7995...",False,False,False,False,False,False,False,False
3,162406236,18,18,Pista Expressa Marg. Tietê,7855,39,1825,"{""type"":""LineString"", ""coordinates"":[[-46.6325...",True,False,False,False,False,False,True,False
4,162400697,8,8,Av. Morvan Dias de Figueiredo,4354,34,2167,"{""type"":""LineString"", ""coordinates"":[[-46.5910...",True,False,False,False,False,False,False,False


In [83]:
irreg_data.to_csv(OUTPUT_PATH/'irreg_jams_by_region.csv')

## Maior comprimento de lentião irregular nas vias contidas em cada região e em toda cidade

### Por Região Analisada

In [84]:
jams = {}
for i in range(1,9):
    jams[f'region{i}'] = {}
    jam_reg = irreg_data.loc[irreg_data[f'bbox{i}'], 'min_hour':'driverscount']
    jam_hour = {h: jam_reg.loc[(jam_reg['min_hour'] <= h) & (h <= jam_reg['max_hour']), 'length'].sum() for h in range(-2, 24)}
    jams[f'region{i}']['hour_max_jam'] = max(jam_hour.items(), key=operator.itemgetter(1))[0]
    jams[f'region{i}']['max_jam'] = max(jam_hour.items(), key=operator.itemgetter(1))[1]

In [85]:
for i in range(1,9):
    print(f'total_interactions_reg{i}')

total_interactions_reg1
total_interactions_reg2
total_interactions_reg3
total_interactions_reg4
total_interactions_reg5
total_interactions_reg6
total_interactions_reg7
total_interactions_reg8


In [86]:
jams

{'region1': {'hour_max_jam': 8, 'max_jam': 44732},
 'region2': {'hour_max_jam': 9, 'max_jam': 20856},
 'region3': {'hour_max_jam': 9, 'max_jam': 6819},
 'region4': {'hour_max_jam': 7, 'max_jam': 613},
 'region5': {'hour_max_jam': 10, 'max_jam': 4106},
 'region6': {'hour_max_jam': 8, 'max_jam': 4229},
 'region7': {'hour_max_jam': 8, 'max_jam': 25341},
 'region8': {'hour_max_jam': -2, 'max_jam': 0}}

### Na cidade

In [87]:
jam_reg = irreg_data.loc[:, 'min_hour':'driverscount']
jam_total = {h: jam_reg.loc[(jam_reg['min_hour'] <= h) & (h <= jam_reg['max_hour']), 'length'].sum() for h in range(-2, 24)}

In [88]:
print(max(jam_total.items(), key=operator.itemgetter(1))[0])
print(max(jam_total.items(), key=operator.itemgetter(1))[1])

8
750816


# Lentidão das Vias para Análie Auxiliar

In [89]:
irreg_data.sort_values(by='length', ascending=False)

,id,min_hour,max_hour,street,length,nthumbsup,driverscount,irre_geojson,bbox1,bbox2,bbox3,bbox4,bbox5,bbox6,bbox7,bbox8
127,162400346,6,6,BR-116 Rod. Pres. Dutra,17413,5,644,"{""type"":""LineString"", ""coordinates"":[[-46.3718...",False,False,False,False,False,False,False,False
21,162403443,11,11,Pista Expressa Marg. Tietê,16961,17,2067,"{""type"":""LineString"", ""coordinates"":[[-46.7365...",True,False,False,False,False,False,False,False
7,162421411,8,8,SP-160 N Rod. dos Imigrantes,15385,30,2417,"{""type"":""LineString"", ""coordinates"":[[-46.5955...",False,False,False,False,False,False,False,False
365,162400524,7,8,BR-116 Rod. Pres. Dutra,13409,1,223,"{""type"":""LineString"", ""coordinates"":[[-46.5284...",False,False,False,False,False,False,False,False
24,162404004,12,12,Pista Expressa Marg. Tietê,13190,14,2074,"{""type"":""LineString"", ""coordinates"":[[-46.6810...",True,False,False,False,False,False,True,False
5,162420951,8,8,SP-021 Rodoanel Mário Covas,12970,31,1458,"{""type"":""LineString"", ""coordinates"":[[-46.8296...",False,False,False,False,False,False,False,False
134,162401964,9,9,SP-070 O Rod. Ayrton Senna da Silva,12799,5,458,"{""type"":""LineString"", ""coordinates"":[[-46.4440...",False,False,False,False,False,False,False,False
48,162402177,11,11,SP-070 O Rod. Ayrton Senna da Silva,11983,10,745,"{""type"":""LineString"", ""coordinates"":[[-46.4440...",False,False,False,False,False,False,False,False
8,162400784,8,8,SP-160 N Rod. dos Imigrantes,11867,29,2011,"{""type"":""LineString"", ""coordinates"":[[-46.5955...",False,False,False,False,False,False,False,False
67,162401365,8,8,SP-270 L Rod. Raposo Tavares,11586,8,1078,"{""type"":""LineString"", ""coordinates"":[[-46.9108...",False,False,False,False,False,False,False,False
